HOUSING PRICE USING MLFLOW

In [2]:
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing

In [5]:
housing = fetch_california_housing

In [8]:
#Preparing the dataset
housing_data = housing()
data = pd.DataFrame(housing_data.data, columns=housing_data.feature_names)
data['Price'] = housing_data.target
data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Price
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


Train Test split, Model hyperparameter tuning, Mlfloow experiments

In [9]:
from urllib.parse import urlparse
#Dividing data into independent and dependent features
X = data.drop(columns=["Price"])
y= data["Price"]

In [10]:
#Hyper parameter tuning using Grid SearchCV
def hyper_parameter_tuning(X_train, y_train, param_grid):
    rf = RandomForestRegressor()
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1,verbose=2,scoring=
                               "neg_mean_squared_error")
    grid_search.fit(X_train,y_train)
    return grid_search

In [14]:
#Spliting data into train and test
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
from mlflow.models import infer_signature
signature = infer_signature(X_train, y_train)

## Define the hyper parameter grid
param_grid = {
    'n_estimators': [100,200],
    'max_depth': [5,10,None],
    'min_samples_split': [2,5],
    'min_samples_leaf': [1,2]
}

In [16]:
#Starting mlflow experiment


with mlflow.start_run():
    grid_search = hyper_parameter_tuning(X_train, y_train,param_grid)
    ## Get the best model
    best_model = grid_search.best_estimator_
    #Evaluate the best model
    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test,y_pred)
    ##Log the best parameters and metrics
    mlflow.log_param("best_n_estimators", grid_search.best_params_['n_estimators'])
    mlflow.log_param("best_max_depth", grid_search.best_params_['max_depth'])
    mlflow.log_param("best_min_samples_split", grid_search.best_params_['min_samples_split'])
    mlflow.log_param("best_min_samples_leaf", grid_search.best_params_['min_samples_leaf'])
    mlflow.log_metric("mse", mse)

    #Tracking url
    mlflow.set_tracking_uri("http://127.0.0.1:5000")
    mlflow.set_experiment("Houseing Price")
    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
    if tracking_url_type_store != "file":
        mlflow.sklearn.log_model(best_model,"model", registered_model_name="Best RandomForest Model")
    else:
        mlflow.sklearn.log_model(best_model,"model", signature=signature)
    print(f"Best Hyper parameter is: {grid_search.best_params_}")
    print(f"Mean squared error {mse}")


Fitting 3 folds for each of 24 candidates, totalling 72 fits


2025/01/02 08:27:11 INFO mlflow.tracking.fluent: Experiment with name 'Houseing Price' does not exist. Creating a new experiment.
2025/01/02 08:27:27 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'Best RandomForest Model'.
2025/01/02 08:27:28 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Best RandomForest Model, version 1


Best Hyper parameter is: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
Mean squared error 0.25301251084408566
🏃 View run kindly-gnat-260 at: http://127.0.0.1:5000/#/experiments/0/runs/13c270aaa6d3496380ee4568a68b4127
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0


Created version '1' of model 'Best RandomForest Model'.
